## Spotting Bugs: Part 1

We want to calculate
$$
 I = \sum_{i=0}^{10} f(x_i), \quad \quad x_i = i\times 0.1  
$$
where $x$ varies in steps of 0.1 over the interval $x\in[0,1]$. If we define 
$$
 f(x) = 1
$$
then the sum above will count the number of points $x_i$.

In [1]:
def f(x):
    return( 1 )

# Do a small test
xVec = [0.1*i for i in range(11)] # the x_i
[f(x) for x in xVec] # the terms in the sum

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Now we try the loop from lecture. If it calculated the sum above correctly, it should give 11, as we evaluate $f(x)$ at the 11 points 0, 0.1, 0.2 $\dots$ 1.0, but it doesn't: it gives 12 instead becasue it adds $f(1.1)$ into the sum.

In [2]:
# Try a python version of the loop from the lecture notes
x = 0
sum = f(x)
while(x <= 1):
    x = x + 0.1
    sum = sum + f(x)
    
print( sum )

12


## Spotting Bugs: Part 2

Now we want to calculate
$$
 I_N = \sum_{i=0}^{N} f(x_i), \quad \quad x_i = i\times (1/N)  
$$
so that we take $N$ steps across the closed interval $[0,1]$. 

As above, we'll define 
$
 f(x) = 1
$
so that the sum $I_N$ will count the number of points $x_i$. That is, if $f(x) = 1$ for all $x \in [0,1]$, then $I_N = N + 1.$

In [3]:
# This time we'll package the loop up into a function 
# that we can test for various value of N
def sumLoop(N):
    h = 1/N
    x = 0
    sum = 0
    while(x <= 1):
        sum = sum + f(x)
        x = x + h

    return( sum )

# Compute the sum from Part 1 
sumLoop(10)

11

So the revised loop works for $N = 10$, but, as the next example shows, it doesn't work for all vales of $N$:

In [4]:
sumLoop(100)

100

The problem is that $0.01$ isn't represented exactly by python's `float` type and so the value of `x` on the $j$-th pass through the loop isn't exactly equal to $j \times 0.01$. This is illustrated below, by a loop that prints `x - j/100` when `x` is near 1.0.

In [5]:
# A revised loop that prints output in late stages of te computation
def sumLoopWithVariableStep(N):
    h = 1/N
    x = 0
    sum = 0
    nPasses = 0 # counts passes through the loop
    while(x <= 1):
        sum = sum + f(x)
        x = x + h
        nPasses = nPasses + 1 

        if( x >= 0.9 ):
            print( "nPasses = {0}, x - nPasses/{1} = {2}".format( nPasses, N, x - (nPasses/N) ) )

    return( sum )

# See what happens when N = 100
sumLoopWithVariableStep( 100 )

nPasses = 90, x - nPasses/100 = 5.551115123125783e-16
nPasses = 91, x - nPasses/100 = 5.551115123125783e-16
nPasses = 92, x - nPasses/100 = 5.551115123125783e-16
nPasses = 93, x - nPasses/100 = 5.551115123125783e-16
nPasses = 94, x - nPasses/100 = 6.661338147750939e-16
nPasses = 95, x - nPasses/100 = 6.661338147750939e-16
nPasses = 96, x - nPasses/100 = 6.661338147750939e-16
nPasses = 97, x - nPasses/100 = 6.661338147750939e-16
nPasses = 98, x - nPasses/100 = 6.661338147750939e-16
nPasses = 99, x - nPasses/100 = 6.661338147750939e-16
nPasses = 100, x - nPasses/100 = 6.661338147750939e-16


100

You can see that at the end of the 100th pass through the loop, `x` is just slightly larger than 1.0 and so the test at the top the `while` loop, `(x <= 1)`, fails and we never add the final contribution, which should correspond to $f(x_{100}) = f(1) = 1$. 

#### Fixing the problem

The safest way to manage the loop is to avoid floating-point arithmetic altogether and just count steps:

In [6]:
def sumLoopWithCounter(N):
    h = 1/N
    x = 0
    sum = 0
    nSteps = 0
    while(nSteps <= N):
        sum = sum + f(x)
        x = x + h
        nSteps = nSteps + 1 

    return( sum )

# Compute the sum that went wrong above. 
sumLoopWithCounter(100)

101